In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import dataEngr
reload(dataEngr)

Read in just the testing and training data.

In [ ]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
xgbInput.split_data(update_trainDf=True)

In [1]:
%run -i custom_paramsearch.py

number of columns before one hot encoding 370
number of columns after one hot encoding 469
4

[0]	train-error:0.173140	eval-error:0.146052
[1]	train-error:0.172071	eval-error:0.143392
[2]	train-error:0.170767	eval-error:0.143293
[3]	train-error:0.170628	eval-error:0.142632
[4]	train-error:0.170535	eval-error:0.142416
[5]	train-error:0.170398	eval-error:0.141991
[6]	train-error:0.170497	eval-error:0.142243
[7]	train-error:0.169953	eval-error:0.142394
[8]	train-error:0.169848	eval-error:0.141793
[9]	train-error:0.169781	eval-error:0.142012
[10]	train-error:0.169426	eval-error:0.141202
[11]	train-error:0.169223	eval-error:0.141475
[12]	train-error:0.169139	eval-error:0.141364
[13]	train-error:0.169095	eval-error:0.141599
[14]	train-error:0.168822	eval-error:0.141624
[15]	train-error:0.167517	eval-error:0.140381
[16]	train-error:0.167222	eval-error:0.140098
[17]	train-error:0.166362	eval-error:0.139952
[18]	train-error:0.165743	eval-error:0.139986
[19]	train-error:0.164794	eval-error:0.138919
[20]	train-error:0.164106	eval-error:0.138549
[21]	train-error:0.163962	eval-error:0.13828


5

[0]	train-error:0.173055	eval-error:0.145591
[1]	train-error:0.171646	eval-error:0.144031
[2]	train-error:0.166000	eval-error:0.138479
[3]	train-error:0.165939	eval-error:0.139328
[4]	train-error:0.165929	eval-error:0.138850
[5]	train-error:0.165323	eval-error:0.138220
[6]	train-error:0.165380	eval-error:0.139011
[7]	train-error:0.164744	eval-error:0.137534
[8]	train-error:0.164659	eval-error:0.137694
[9]	train-error:0.164443	eval-error:0.137277
[10]	train-error:0.164198	eval-error:0.136937
[11]	train-error:0.163924	eval-error:0.136521
[12]	train-error:0.163837	eval-error:0.137108
[13]	train-error:0.163597	eval-error:0.137227
[14]	train-error:0.163481	eval-error:0.136441
[15]	train-error:0.163344	eval-error:0.136468
[16]	train-error:0.163214	eval-error:0.136631
[17]	train-error:0.163085	eval-error:0.136630
[18]	train-error:0.162947	eval-error:0.136309
[19]	train-error:0.162834	eval-error:0.136268
[20]	train-error:0.162747	eval-error:0.136393
[21]	train-error:0.162701	eval-error:0.13646

In [4]:
xgbInput.train_Y

array([ 7,  7, 10, ...,  7,  7,  7])

In [ ]:
def cv_sessions(xgbInput):
    """Cross-validation on a subset of training data which meets a condition"""
    
    condition = 'dac_year == 2014 & action_counts != -1'
    idx = np.nonzero(xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index))
    idx = idx[0]
    np.random.shuffle(idx[0])
    n = len(idx)
    
    for i in range(0, kfolds):
        idx[0]
        
def cv_bydate(xgbInput):
    """Select folds for cross validation as all cases that occurred in a given with month 
    in 2014, with sessions data.
    Only cases in 2014 have sessions data and the last test case is on June 30.
    """
    
    for i in range(4,7):
        condition = 'dac_year == 2014 & dac_month == @i & action_counts != -1'
        valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)
        valid_indx = np.where(valid_mask)[0]
        train_indx = np.where(~valid_mask)[0]
        yield train_indx, valid_indx

In [ ]:
pd.read_pickle('cv_results/sessions_e20_25n/validate_err.p')

In [ ]:
a = pd.read_pickle('cv_results/actions_e20/validate_err.p')
a = a.astype(float)
a['mean'] = a.mean(axis = 1)

In [ ]:
%run run_model.py

In [ ]:
with open('../xgbmodels/actions2_e16_53n.p') as f:
    bst = pickle.load(f)

In [ ]:
xgbInput.allDf.columns[xgbInput.allDf.columns.str.contains('book')]

In [ ]:
len(features)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]